In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np



In [71]:
# save filepath 
home_data_file_path = '/Users/balqeesjabri/Downloads/train.csv'
# read the data and store data in DataFrame home_data
home_data = pd.read_csv(home_data_file_path)

In [72]:
# target
y = home_data.SalePrice
X = home_data.drop(['SalePrice'], axis=1)
# Divide data into training and validation subsets
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y,
                                                                random_state=0)

In [73]:
# Select categorical columns 
categorical_cols = [colname for colname in X_train_full.columns if X_train_full[colname].dtype == "object"]

# Select numerical columns
numerical_cols = [colname for colname in X_train_full.columns if X_train_full[colname].dtype in ['int64', 'float64']]

# Keep selected columns only
my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [76]:
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.decomposition import PCA
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

#Create transformers for numerical features 
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
   
])

# Create transformers for categorical features
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Create a ColumnTransformer to apply different transformers to both feature types
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_transformer, numerical_cols),
    ('cat', categorical_transformer, categorical_cols)
])



In [77]:
from xgboost import XGBRegressor
# define model 
my_model = XGBRegressor(n_estimators=900, learning_rate=0.07)

1. XGBRegressor(n_estimators=900, learning_rate=0.07): rmse: 29988.874835543847

In [78]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

# create a pipline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', my_model)
                             ])
# This line fits the pipeline to the training data,
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
rmse = np.sqrt(mean_squared_error(y_valid, preds))
print('rmse:', rmse)


rmse: 29988.999361368544


In [79]:
# Preprocess the data using the preprocessor
X_train_preprocessed = preprocessor.transform(X_train)
X_valid_preprocessed = preprocessor.transform(X_valid)

In [80]:
from sklearn.decomposition import PCA

pca = PCA(n_components=None)
pca.fit(X_train_preprocessed)

exp_var = pca.explained_variance_ratio_ * 100
cum_exp_var = np.cumsum(exp_var)
plt.figure(figsize=(15, 10))
plt.bar(range(1, 81), exp_var, align='center',
        label='Individual explained variance', width=0.5)
plt.step(range(1, 81), cum_exp_var, where='mid',
         label='Cumulative explained variance', color='red')

plt.ylabel('Explained variance percentage')
plt.xlabel('Principal component index')
plt.xticks(ticks=list(range(1, 81)))
plt.legend(loc='best')
plt.tight_layout()

plt.savefig("cumulative_explained_variance_plot.png")

PCA()

In [81]:
from sklearn.decomposition import PCA
# we can now specify the number of components 
pca = PCA(n_components=49)
X_pca = pd.DataFrame(pca.fit_transform(X_train_preprocessed)
)
# Split the PCA-transformed data 
X_train_pca, X_valid_pca, y_train, y_valid = train_test_split(X_pca, y_train, random_state=50)

In [96]:
# define model
my_model = XGBRegressor(n_estimators=900, learning_rate=0.07)
# fit model
my_model.fit(X_train_pca, y_train, 
             early_stopping_rounds=7, 
             eval_set=[(X_valid_pca, y_valid)], 
             verbose=False)

/usr/local/lib/python3.11/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.07, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=900, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [97]:
# Make predictions on the test data
y_pred = my_model.predict(X_valid_pca)

In [99]:
# Evaluate the model's performance using  regression metrics

score = mean_absolute_error(y_valid, y_pred)
rmse = np.sqrt(mean_squared_error(y_valid, y_pred))

print("Mean Absolute Error (MAE):", score)
print("Root Mean Squared Error (RMSE):", rmse)

Mean Absolute Error (MAE): 18972.35933223084
Root Mean Squared Error (RMSE): 27437.643154524903
